In [84]:
import pandas as pd
from scipy import sparse

In [145]:
ratings = pd.read_csv('dataset/ratings.csv')
movies = pd.read_csv('dataset/movies.csv')
all_movie = movies['title'].values
new_movie = []
for movie in all_movie:
    split_movie = movie.split()
    split_movie.pop()   
    string = ' '.join(split_movie)
    new_movie.append(string)
movies['movie'] = new_movie
movies.head()

,movieId,title,genres,movie
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II


In [147]:
ratings = pd.merge(movies,ratings).drop(['title','genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,movieId,movie,userId,rating
0,1,Toy Story,1,4.0
1,1,Toy Story,5,4.0
2,1,Toy Story,7,4.5
3,1,Toy Story,15,2.5
4,1,Toy Story,17,4.5


In [149]:
userRatings = ratings.pivot_table(index=['userId'],columns=['movie'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9444)
After:  (610, 2265)


In [150]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

movie,"'burbs, The",(500) Days of Summer,10 Cloverfield Lane,10 Things I Hate About You,"10,000 BC",101 Dalmatians,101 Dalmatians (One Hundred and One Dalmatians),12 Angry Men,12 Years a Slave,127 Hours,...,Zack and Miri Make a Porno,Zero Dark Thirty,Zero Effect,Zodiac,Zombieland,Zoolander,Zootopia,eXistenZ,xXx,¡Three Amigos!
movie,,,,,,,,,,,,,,,,,,,,,
"'burbs, The",1.000000,0.063117,-0.023768,0.143482,0.011998,0.087931,0.224052,0.034133,0.009277,0.008331,...,0.017477,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,0.187953,0.062174,0.353194
(500) Days of Summer,0.063117,1.000000,0.142471,0.273989,0.193960,0.148903,0.142141,0.160679,0.135486,0.200135,...,0.374515,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.053614,0.241092,0.125905
10 Cloverfield Lane,-0.023768,0.142471,1.000000,-0.005799,0.112396,0.006139,-0.016835,0.031619,-0.024275,0.272943,...,0.242663,0.099059,-0.023477,0.272347,0.241751,0.195054,0.319371,0.177846,0.096638,0.002733
10 Things I Hate About You,0.143482,0.273989,-0.005799,1.000000,0.244670,0.223481,0.211473,0.013083,0.091964,0.043383,...,0.243118,0.104858,0.132460,0.091853,0.158637,0.281934,0.050031,0.121029,0.130813,0.110612
"10,000 BC",0.011998,0.193960,0.112396,0.244670,1.000000,0.234459,0.119132,0.059068,-0.025882,0.089328,...,0.260261,0.087592,0.094913,0.184521,0.242299,0.240231,0.094773,0.088045,0.203002,0.083518
101 Dalmatians,0.087931,0.148903,0.006139,0.223481,0.234459,1.000000,0.285112,0.120578,0.072399,0.029967,...,0.114968,0.077232,0.096294,0.067134,0.113224,0.184324,0.054024,0.047804,0.156932,0.078734
101 Dalmatians (One Hundred and One Dalmatians),0.224052,0.142141,-0.016835,0.211473,0.119132,0.285112,1.000000,0.135603,0.017264,-0.046277,...,0.120302,0.125816,0.049818,0.083650,0.171654,0.274260,0.077594,0.085606,0.248820,0.171118
12 Angry Men,0.034133,0.160679,0.031619,0.013083,0.059068,0.120578,0.135603,1.000000,0.135010,0.058741,...,0.104341,0.030179,0.079764,0.241083,0.145856,0.121873,0.056603,-0.001768,0.074160,0.102563
12 Years a Slave,0.009277,0.135486,-0.024275,0.091964,-0.025882,0.072399,0.017264,0.135010,1.000000,0.249931,...,0.024045,0.038127,0.013786,0.190366,0.104150,0.017351,0.063325,0.002528,0.037469,0.004213


In [ ]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [152]:
romantic_lover = [("(500) Days of Summer",5)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)
similar_movies.head(10)

,"'burbs, The",(500) Days of Summer,10 Cloverfield Lane,10 Things I Hate About You,"10,000 BC",101 Dalmatians,101 Dalmatians (One Hundred and One Dalmatians),12 Angry Men,12 Years a Slave,127 Hours,...,Zack and Miri Make a Porno,Zero Dark Thirty,Zero Effect,Zodiac,Zombieland,Zoolander,Zootopia,eXistenZ,xXx,¡Three Amigos!
0,0.157792,2.5,0.356179,0.684973,0.4849,0.372257,0.355353,0.401698,0.338715,0.500338,...,0.936288,0.446637,0.171018,1.036463,0.889309,0.630565,0.540017,0.134034,0.60273,0.314763


In [153]:
similar_movies.sum().sort_values(ascending=False)

(500) Days of Summer           2.500000
Silver Linings Playbook        1.254898
Adventureland                  1.157021
Up in the Air                  1.129961
50/50                          1.123890
                                 ...   
Stargate                      -0.142870
Madness of King George, The   -0.144495
Disclosure                    -0.160575
Postman, The (Postino, Il)    -0.161373
Clear and Present Danger      -0.209810
Length: 2265, dtype: float64

In [159]:
get_similar("Inception",5).index

Index(['Inception', 'Dark Knight, The', 'Inglourious Basterds',
       'Shutter Island', 'Dark Knight Rises, The', 'Interstellar', 'Up',
       'Django Unchained', 'Avengers, The', 'Social Network, The',
       ...
       'Cliffhanger', 'Madness of King George, The', 'Piano, The',
       'Star Trek: Generations', 'Firm, The', 'Crimson Tide', 'True Lies',
       'Disclosure', 'Get Shorty', 'Clear and Present Danger'],
      dtype='object', name='movie', length=2265)

In [117]:
# split_list = list[1].split()
# split_list.pop()
# split_list

['Alien']

In [125]:
# all_movie = movies['title'].values
# all_movie

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'Flint (2017)', 'Bungo Stray Dogs: Dead Apple (2018)',
       'Andrew Dice Clay: Dice Rules (1991)'], dtype=object)

In [158]:
# new_movie = []
# for movie in all_movie:
#     split_movie = movie.split()
#     split_movie.pop()   
#     string = ' '.join(split_movie)
#     new_movie.append(string)
